In [1]:
import pandas as pd

In [47]:
def segment_data(df):
    pos_dict = {}
    pos = ['PG','SG','PF','SF','C','F','G']
    
    for p in pos:
        pos_dict[p] = df.loc[df['POS'] == p]
        
    return pos_dict

In [85]:
def get_data(last_n_years,season):
    from time import strftime, localtime
    import pandas as pd
    
    this_year = int(strftime("%Y",localtime())) -1
    dfs = {}
    for y in range(this_year,this_year - last_n_years,-1):
        df = pd.read_csv("./player_stats/"+str(y)+"/"+season+".csv")
        dfs[y] = segment_data(df)
        
    return dfs

In [86]:
def player_slider():
    slider = dcc.Slider(
                        id = 'n_players',
                        min = 1,
                        max = 10,
                        value = 5,
                        marks = {str(n): str(n) for n in range(1,11)},
                        step = None
                    )
    return slider

In [87]:
def year_slider():
    slider = dcc.Slider(
                        id = 'years',
                        min = min(dfs.keys()),
                        max = max(dfs.keys()),
                        value = max(dfs.keys()),
                        marks = {year: str(year) for year in dfs.keys()},
                        step = None
                    )
    return slider

In [88]:
def player_stats():
    stats = dcc.RadioItems(
                            options=[
                                {'label': 'PTS','value':'PTS'},
                                {'label': 'AST','value':'AST'},
                                {'label': 'REB','value':'REB'},
                                {'label': 'BLK','value':'BLK'},
                                {'label': 'STL','value':'STL'}
                            ],
                            value = 'PTS',
                            id = "stats"
                    )
    return stats

In [177]:
def pos_dropdown():
    d = dcc.Dropdown(
                    options = [
                        {"value":"PG", "label":"Point Guard"},
                        {"value":"SG", "label":"Shooting Guard"},
                        {"value":"G", "label":"Guard"},
                        {"value":"PF", "label":"Power Forward"},
                        {"value":"SF", "label":"Small Forward"},
                        {"value":"F", "label":"Forward"},
                        {"value":"C", "label":"Centre"}
                    ],
                    value = "PG",
                    id = "pos_drop"
                )
    return d

In [178]:
def season_dropdown():
    d = dcc.Dropdown(
                    options = [
                        {"value":"regular", "label":"Regular"},
                        {"value":"post", "label":"Post"}
                    ],
                    value = "regular",
                    id = "season_drop"
                )
    return d

In [179]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import date

season_types = ["regular_season","post_season"]
r_season = season_types[0]
p_season = season_types[1]


y = 2021
pos = 'PG'
top_n_players = 5

reg_season = get_data(5,r_season)
post_season = get_data(5,p_season)

seasons = {"regular":reg_season, "post":post_season}

sorted_df = seasons["regular"][y][pos].sort_values("PTS",ascending = False).head(top_n_players)

fig = px.bar(sorted_df, x='PTS', y="Name", color="Team", text = 'PTS',
             title="PTS for top "+str(top_n_players)+" players at "+ pos + " in the "+str(y) +" regular season")




app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#777777'}
text_size = {'H1':48,'H2':40,'text':28}

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'calibri'}, children=[

    html.H1(children='NBA Data visualisation',
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),


    html.Div(children=[
        
            html.Div([html.Label("Position: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        pos_dropdown()
                    ],
                style = {'textAlign': 'left',"flex":1}
            ),
        
            html.Div([html.Label("Season: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        season_dropdown()
                     ],
                style = {'textAlign': 'center',"flex":1}
            ),
        
#             html.Div([season_dropdown()], style = {'textAlign': 'center'}),
        
            html.Div([html.Label("Stats", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        player_stats()
                     ],
                style = {'textAlign': 'right',"flex":1}
            )
        
        
            

        ],style = {'display':'flex','flex-direction': 'row'}
    ),
    
    player_slider(),
    
    dcc.Graph(figure = fig, id = 'graph'),
    
    year_slider()
])

@app.callback(
    Output('graph','figure'),
    Input('stats','value'),
    Input('years','value'),
    Input('pos_drop','value'),
    Input('n_players','value'),
    Input('season_drop','value'))
def update_figure(stat,y,pos,top_n_players,season_type):
    
    sorted_df = seasons[season_type][y][pos].sort_values(stat,ascending = False).head(top_n_players)
    fig = px.bar(sorted_df, x=stat, y='Name', color="Team",text = stat,
                    title="PTS for top "+str(top_n_players)+" players at "+ pos + " in the "+str(y)+\
                            " "+ season_type + " season")

    return fig

In [181]:
app.run_server(mode = "inline")

/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

